In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
%matplotlib notebook

# 用Iris数据集进行理论演示

In [ ]:
from sklearn import datasets #导入sklearn中的datasets模块，方便下载内置的iris数据集
#导入数据
x_data = datasets.load_iris().data #导入iris数据集的特征
y_data = datasets.load_iris().target #导入iris数据集的标签

In [ ]:
plt.scatter(x_data[:,0], x_data[:,2], c = y_data)
plt.xlabel('sepal length (cm)')
plt.ylabel('petal length (cm)')

In [ ]:
x_data = np.vstack([x_data[:,0],x_data[:,2]]).T

## 1对1

### 划分数据集和训练

In [ ]:
# 划分数据集
one_vs_one_dataset = {}
for i in range(3):
    for j in range(3):
        if i >= j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            idx1 = y_data == i
            idx2 = y_data == j
            x_data_i = x_data[idx1]
            x_data_j = x_data[idx2]
            y_data_i = y_data[idx1]
            y_data_j = y_data[idx2]
            y_data_i = np.zeros_like(y_data_i)
            y_data_j = np.ones_like(y_data_j)
            x_data_ = np.concatenate([x_data_i,x_data_j], axis=0)
            y_data_ = np.concatenate([y_data_i,y_data_j], axis=0)
            x_data_ = tf.cast(x_data_,dtype=tf.float32)
            one_vs_one_dataset[id_] = (x_data_, y_data_)

In [ ]:
len(one_vs_one_dataset)

In [ ]:
class LogisticRegression(tf.keras.Model):
    
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(1)
        
    def __call__(self, x, training=None):
        logit = self.dense(x)
        y_hat = tf.math.sigmoid(logit)
        return y_hat

In [ ]:
one_vs_one_lr = {}
for i in range(3):
    for j in range(3):
        if i >= j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            one_vs_one_lr[id_] = LogisticRegression()

In [ ]:
historys = []
for i in range(3):
    for j in range(3):
        if i >= j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            lr_model = one_vs_one_lr[id_]
            x, y  = one_vs_one_dataset[id_]
            
            x = tf.convert_to_tensor(x, dtype=tf.float32)
            y = tf.convert_to_tensor(y, dtype=tf.float32)

            lr_model.compile(optimizer='adam',
                             loss='binary_crossentropy', # 这里用二元的交叉熵作为二分类的损失函数
                             metrics=['acc'] # 在训练时输出accuracy(精度,即正确率)
                            )
            
            historys.append(lr_model.fit(x, y, epochs=100, batch_size =16, verbose=0)) # verbose=0不从std输出,不然导出markdown这块就太长了

In [ ]:
x_axis = range(100)
fig, ax = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=False)
ax = ax.flatten()
for i in range(3):
    loss = historys[i].history['loss']
    ax[i].plot(x_axis, loss)
    ax[i].set_title('classifier {} loss curve'.format(i+1))
for i in range(3):
    acc  = historys[i].history['acc']
    ax[i+3].plot(x_axis, acc)
    ax[i+3].set_title('classifier {} acc  curve'.format(i+1))


ax[0].set_ylabel('loss')
ax[3].set_ylabel('acc')
plt.show()

### 可视化 

In [ ]:
ids_ = []
for i in range(3):
    for j in range(3):
        if i >= j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            ids_.append(id_)

In [ ]:
id_ = ids_[2]
lr_model = one_vs_one_lr[id_]
x, y  = one_vs_one_dataset[id_]
x_min, x_max = x.numpy()[:, 0].min() - 1, x.numpy()[:, 0].max() + 1
y_min, y_max = x.numpy()[:, 1].min() - 1, x.numpy()[:, 1].max() + 1

y_hat = x.numpy().dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()

In [ ]:
colors = [plt.cm.BuGn, plt.cm.BuPu, plt.cm.PuBuGn,
         plt.cm.BuGn_r, plt.cm.BuPu_r, plt.cm.PuBuGn_r]

In [ ]:
def plot_boundary(id_):
    plt.figure(figsize=(5,5))
    lr_model = one_vs_one_lr[id_]
    x, y  = one_vs_one_dataset[id_]

    x_min, x_max = 0, 8
    y_min, y_max = 0, 10

    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    z_hat = np.vstack([xx.ravel(), yy.ravel()]).T.dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()
    z_hat = tf.math.sigmoid(z_hat.ravel())
    z_hat = z_hat.numpy() > 0.5
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=colors[i])
    plt.scatter(x.numpy()[:, 0], x.numpy()[:, 1], c=y, cmap=colors[i+3])

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('classifier {}'.format(id_))

In [ ]:
plot_boundary('0v1')

In [ ]:
plot_boundary('0v2')

In [ ]:
plot_boundary('1v2')

In [ ]:
x_data.shape[0]

In [ ]:
def predict_multi_label(x_data, one_vs_one_lr):
    votes = {i:np.zeros([x_data.shape[0],1]) for i in range(3)}
    for i in range(3):
        for j in range(3):
            if i >= j:
                continue
            else:
                id_ = str(i)+'v'+str(j)
                lr_model = one_vs_one_lr[id_]
                y_hat = x_data.dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()
                y_hat = tf.math.sigmoid(y_hat)
                votes[i] += (tf.cast(y_hat <= 0.5, dtype=tf.int32).numpy())
                votes[j] += (tf.cast(y_hat > 0.5, dtype=tf.int32).numpy())
    
    votes = np.hstack([votes[0],votes[1],votes[2]])
    y_hat = np.argmax(votes, axis=-1)
    return y_hat
                
predict_multi_label(x_data, one_vs_one_lr)  

In [ ]:
def plot_boundary_multi(one_vs_one_lr):
       
    plt.figure(figsize=(5,5))
    x_min, x_max = 0, 8
    y_min, y_max = 0, 10
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    
    z_hat = predict_multi_label(np.vstack([xx.ravel(), yy.ravel()]).T, one_vs_one_lr)  
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlGn)
    plt.scatter(x_data[:, 0], x_data[:, 1], c=y_data, cmap=plt.cm.YlOrBr)

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('multi classifier')

In [ ]:
plot_boundary_multi(one_vs_one_lr)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_data, predict_multi_label(x_data, one_vs_one_lr))

In [ ]:
cm

## 一对多

In [ ]:
# 划分数据集
one_vs_rest_dataset = {}
for i in range(3):
    id_ = str(i)
    idx1 = y_data == i
    idx2 = y_data != i
    x_data_i = x_data[idx1]
    x_data_j = x_data[idx2]
    y_data_i = y_data[idx1]
    y_data_j = y_data[idx2]
    y_data_i = np.zeros_like(y_data_i)
    y_data_j = np.ones_like(y_data_j)
    x_data_ = np.concatenate([x_data_i,x_data_j], axis=0)
    y_data_ = np.concatenate([y_data_i,y_data_j], axis=0)
    x_data_ = tf.cast(x_data_,dtype=tf.float32)
    one_vs_rest_dataset[id_] = (x_data_, y_data_)

In [ ]:
one_vs_rest_lr = {}
for i in range(3):
    id_ = str(i)
    one_vs_rest_lr[id_] = LogisticRegression()

In [ ]:
historys = []
for i in range(3):
    id_ = str(i)
    lr_model = one_vs_rest_lr[id_]
    x, y  = one_vs_rest_dataset[id_]

    x = tf.convert_to_tensor(x, dtype=tf.float32)
    y = tf.convert_to_tensor(y, dtype=tf.float32)

    lr_model.compile(optimizer='adam',
                     loss='binary_crossentropy', # 这里用二元的交叉熵作为二分类的损失函数
                     metrics=['acc'] # 在训练时输出accuracy(精度,即正确率)
                    )

    historys.append(lr_model.fit(x, y, epochs=100, verbose=0)) 

In [ ]:
x_axis = range(100)
fig, ax = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=False)
ax = ax.flatten()
for i in range(3):
    loss = historys[i].history['loss']
    ax[i].plot(x_axis, loss)
    ax[i].set_title('classifier {} loss curve'.format(i+1))
for i in range(3):
    acc  = historys[i].history['acc']
    ax[i+3].plot(x_axis, acc)
    ax[i+3].set_title('classifier {} acc  curve'.format(i+1))


ax[0].set_ylabel('loss')
ax[3].set_ylabel('acc')
plt.show()

### 可视化 

In [ ]:
ids_ = []
for i in range(3):
    id_ = str(i)
    ids_.append(id_)

In [ ]:
def plot_boundary(id_):
    plt.figure(figsize=(5,5))
    lr_model = one_vs_rest_lr[id_]
    x, y  = one_vs_rest_dataset[id_]

    x_min, x_max = 0, 8
    y_min, y_max = 0, 10

    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    z_hat = np.vstack([xx.ravel(), yy.ravel()]).T.dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()
    z_hat = tf.math.sigmoid(z_hat.ravel())
    z_hat = z_hat.numpy() > 0.5
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=colors[i])
    plt.scatter(x.numpy()[:, 0], x.numpy()[:, 1], c=y, cmap=colors[i+3])

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('classifier {} vs rest'.format(id_))

In [ ]:
plot_boundary(ids_[2])

In [ ]:
def predict_multi_label(x_data, one_vs_rest_lr):
    votes = {i:np.zeros([x_data.shape[0],1]) for i in range(3)}
    for i in range(3):
        id_ = str(i)
        lr_model = one_vs_rest_lr[id_]
        y_hat = x_data.dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()
        y_hat = tf.math.sigmoid(y_hat)
        votes[i] +=  (1 - y_hat)

    votes = np.hstack([votes[0],votes[1],votes[2]])
    y_hat = np.argmax(votes, axis=-1)
    return y_hat
                
predict_multi_label(x_data, one_vs_rest_lr)  

In [ ]:
def plot_boundary_multi(one_vs_rest_lr):
       
    plt.figure(figsize=(5,5))
    x_min, x_max = 0, 8
    y_min, y_max = 0, 10
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    
    z_hat = predict_multi_label(np.vstack([xx.ravel(), yy.ravel()]).T, one_vs_rest_lr)  
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlGn)
    plt.scatter(x_data[:, 0], x_data[:, 1], c=y_data, cmap=plt.cm.YlOrBr)

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('multi classifier')

In [ ]:
plot_boundary_multi(one_vs_rest_lr)

In [ ]:
cm = confusion_matrix(y_data, predict_multi_label(x_data, one_vs_rest_lr))

In [ ]:
cm

## softmax 

In [ ]:
class LogisticRegressionPlus(tf.keras.Model):
    
    def __init__(self, num_of_labels):
        super(LogisticRegressionPlus, self).__init__()
        self.dense = tf.keras.layers.Dense(num_of_labels)
        
    def __call__(self, x, training=None):
        logit = self.dense(x)
        y_hat = tf.math.softmax(logit)
        return y_hat

In [ ]:
LR_plus = LogisticRegressionPlus(3)

In [ ]:
LR_plus.compile(optimizer='adam',loss='sparse_categorical_crossentropy', # 这里用二元的交叉熵作为二分类的损失函数
                metrics=['acc'] # 在训练时输出accuracy(精度,即正确率)
               )

In [ ]:
history = LR_plus.fit(x_data, y_data, epochs=100, verbose = 1)

In [ ]:
x_axis = range(100)
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=False)
ax = ax.flatten()

loss = history.history['loss']
ax[0].plot(x_axis, loss)
ax[0].set_title('softmax classifier loss curve')

acc  = history.history['acc']
ax[1].plot(x_axis, acc)
ax[1].set_title('softmax classifier acc  curve')


ax[0].set_ylabel('loss')
ax[1].set_ylabel('acc')
plt.show()

In [ ]:
def plot_boundary_multi():
       
    plt.figure(figsize=(5,5))
    x_min, x_max = 0, 8
    y_min, y_max = 0, 10
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    
    z_hat = np.argmax(LR_plus(np.vstack([xx.ravel(), yy.ravel()]).T).numpy(),axis=-1)
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlGn)
    plt.scatter(x_data[:, 0], x_data[:, 1], c=y_data, cmap=plt.cm.YlOrBr)

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('softmax classifier')

In [ ]:
plot_boundary_multi()

In [ ]:
confusion_matrix(y_data, np.argmax(LR_plus(x_data).numpy(),axis=-1))

In [ ]:
LR_plus.weights[1].numpy()

In [ ]:
def plot_boundary(id_):
    plt.figure(figsize=(5,5))
    x, y  = one_vs_one_dataset[id_]
    i,j = map(int, id_.split('v'))
    x_min, x_max = 0, 8
    y_min, y_max = 0, 10

    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    w = LR_plus.weights[0].numpy()[:,i] - LR_plus.weights[0].numpy()[:,j]
    b = LR_plus.weights[1].numpy()[i] - LR_plus.weights[1].numpy()[j]
    z_hat = np.vstack([xx.ravel(), yy.ravel()]).T.dot(w) + b
    
    z_hat = tf.math.sigmoid(z_hat.ravel())
    z_hat = z_hat.numpy() < 0.5
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=colors[0])
    plt.scatter(x.numpy()[:, 0], x.numpy()[:, 1], c=y, cmap=colors[3])

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('classifier softmax {}'.format(id_))

In [ ]:
plot_boundary('0v1')

In [ ]:
def predict_multi_label(x_data):
    votes = {i:np.zeros([x_data.shape[0],]) for i in range(3)}
    for i in range(3):
        for j in range(3):
            if i >= j:
                continue
            else:
                w = LR_plus.weights[0].numpy()[:,i] - LR_plus.weights[0].numpy()[:,j]
                b = LR_plus.weights[1].numpy()[i] - LR_plus.weights[1].numpy()[j]
                
                y_hat = x_data.dot(w) + b
                y_hat = tf.math.sigmoid(y_hat)
                # print(y_hat)
                
                votes[i] += (tf.cast(y_hat > 0.5, dtype=tf.int32).numpy())
                votes[j] += (tf.cast(y_hat <= 0.5, dtype=tf.int32).numpy())
    
    votes = np.vstack([votes[0],votes[1],votes[2]])
    # print(votes)
    y_hat = np.argmax(votes, axis=0)
    return y_hat
                
predict_multi_label(x_data)  

In [ ]:
def plot_boundary_multi():
       
    plt.figure(figsize=(5,5))
    x_min, x_max = 0, 8
    y_min, y_max = 0, 10
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))
    
    z_hat = predict_multi_label(np.vstack([xx.ravel(), yy.ravel()]).T)  
    Z = z_hat.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlGn)
    plt.scatter(x_data[:, 0], x_data[:, 1], c=y_data, cmap=plt.cm.YlOrBr)

    plt.xlabel('Sepal Length')
    plt.ylabel('Petal length')
    plt.xlim(0, 8)
    plt.ylim(0, 10)
    plt.title('softmax 1v1 classifier')

In [ ]:
plot_boundary_multi()

In [ ]:
confusion_matrix(y_data, predict_multi_label(x_data))

# MINST

In [ ]:
data = np.load('TibetanMNIST.npz')
X = data['image']
y = data['label']

In [ ]:
fig, ax = plt.subplots(
    nrows=2,
    ncols=5,
    sharex=True,
    sharey=True, )

ax = ax.flatten()
for i in range(10):
    img = X[y == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Blues_r', interpolation='nearest')
    ax[i].set_title(str(i))

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
count = []
for i in range(10):
    count.append(sum(y == i))

f, ax1 = plt.subplots(1, 1, figsize=(7, 3), sharex=True)

x = np.array(list("0123456789"))
y1 = np.array(count)
sns.barplot(x=x, y=y1, palette="rocket", ax=ax1)
ax1.axhline(0, color="k", clip_on=False)
ax1.set_ylabel("frequency of occurrence")

In [ ]:
x = [xx.reshape(28*28) for xx in X]

x_train,x_test,y_data,y_data_test = train_test_split(x,y,test_size=0.3,train_size=0.7,random_state=0)
sc = StandardScaler()
sc.fit(x_train)  # 计算均值和方差
x_data = sc.transform(x_train) #利用计算好的方差和均值进行Z分数标准化
x_data_test = sc.transform(x_test)

In [ ]:
# 划分数据集
one_vs_one_dataset = {}
for i in range(10):
    for j in range(10):
        if i >= j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            idx1 = y_data == i
            idx2 = y_data == j
            x_data_i = x_data[idx1]
            x_data_j = x_data[idx2]
            y_data_i = y_data[idx1]
            y_data_j = y_data[idx2]
            y_data_i = np.zeros_like(y_data_i)
            y_data_j = np.ones_like(y_data_j)
            x_data_ = np.concatenate([x_data_i,x_data_j], axis=0)
            y_data_ = np.concatenate([y_data_i,y_data_j], axis=0)
            x_data_ = tf.cast(x_data_,dtype=tf.float32)
            one_vs_one_dataset[id_] = (x_data_, y_data_)

In [ ]:
class LogisticRegression(tf.keras.Model):
    
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(1)
        
    def __call__(self, x, training=None):
        logit = self.dense(x)
        y_hat = tf.math.sigmoid(logit)
        return y_hat

In [ ]:
one_vs_one_lr = {}
for i in range(10):
    for j in range(10):
        if i == j:
            continue
        else:
            id_ = str(i)+'v'+str(j)
            one_vs_one_lr[id_] = LogisticRegression()

In [ ]:
import time
start = time.time()
historys = []
times = []
accs = []
for i in range(10):
    for j in range(10):
        if i >= j:
            continue
        else:
            
            start_ = time.time()
            id_ = str(i)+'v'+str(j)
            print('在训练',id_)
            lr_model = one_vs_one_lr[id_]
            x, y  = one_vs_one_dataset[id_]
            
            x = tf.convert_to_tensor(x, dtype=tf.float32)
            y = tf.convert_to_tensor(y, dtype=tf.float32)

            lr_model.compile(optimizer='adam',
                             loss='binary_crossentropy', # 这里用二元的交叉熵作为二分类的损失函数
                             metrics=['acc'] # 在训练时输出accuracy(精度,即正确率)
                            )
            
            historys.append(lr_model.fit(x, y, epochs=10, batch_size =16, verbose=0)) 
            times.append(time.time()-start_)
            accs.append(sum(lr_model.predict(x_data_test) == y_data_test)/len(y_data_test))
            # verbose=0不从std输出,不然导出markdown这块就太长了


time_count = time.time() - start

In [ ]:
time_count

In [ ]:
times

In [ ]:
time_count/45

In [ ]:
x_axis = range(10)

fig, ax = plt.subplots(figsize=(20,36), nrows=9, ncols=5, sharex=True, sharey=False)
ax = ax.flatten()
for i in range(45):
    acc = historys[i].history['acc']
    ax[i].plot(x_axis, acc)
    ax[i].set_title('clf {} acc'.format(i+1))

plt.savefig('jinx.jpg')

In [ ]:
x_axis = range(10)

fig, ax = plt.subplots(figsize=(20,36), nrows=9, ncols=5, sharex=True, sharey=False)
ax = ax.flatten()
for i in range(45):
    loss = historys[i].history['loss']
    ax[i].plot(x_axis, loss)
    ax[i].set_title('clf {} loss'.format(i+1))

plt.savefig('jinx1.jpg')

In [ ]:
accs = []
for i in range(10):
    for j in range(10):
        if i >= j:
            continue
        else:

            id_ = str(i)+'v'+str(j)
            print('在测试',id_)
            lr_model = one_vs_one_lr[id_]

            accs.append(sum(lr_model.predict(x_data_test) == y_data_test)/len(y_data_test))
            # verbose=0不从std输出,不然导出markdown这块就太长了

In [ ]:
def predict_multi_label(x_data, one_vs_one_lr):
    votes = {i:np.zeros([x_data.shape[0],1]) for i in range(10)}
    for i in range(10):
        for j in range(10):
            if i >= j:
                continue
            else:
                id_ = str(i)+'v'+str(j)
                lr_model = one_vs_one_lr[id_]
                y_hat = x_data.dot(lr_model.weights[0].numpy()) + lr_model.weights[1].numpy()
                y_hat = tf.math.sigmoid(y_hat)
                votes[i] += (tf.cast(y_hat <= 0.5, dtype=tf.int32).numpy())
                votes[j] += (tf.cast(y_hat > 0.5, dtype=tf.int32).numpy())
    
    votes = np.hstack([votes[0],votes[1],votes[2],votes[3],votes[4],votes[5],votes[6],votes[7],votes[8],votes[9]])
    y_hat = np.argmax(votes, axis=-1)
    return y_hat

In [ ]:
cm = confusion_matrix(y_data, predict_multi_label(x_data, one_vs_one_lr))

In [ ]:
cm

In [ ]:
12097/12476

In [ ]:
class LogisticRegressionPlus(tf.keras.Model):
    
    def __init__(self, num_of_labels):
        super(LogisticRegressionPlus, self).__init__()
        self.dense = tf.keras.layers.Dense(num_of_labels)
        
    def __call__(self, x, training=None):
        logit = self.dense(x)
        y_hat = tf.math.softmax(logit)
        return y_hat

In [ ]:
LR_plus = LogisticRegressionPlus(10)

In [ ]:
LR_plus.compile(optimizer='adam',loss='sparse_categorical_crossentropy', # 这里用二元的交叉熵作为二分类的损失函数
                metrics=['acc'] # 在训练时输出accuracy(精度,即正确率)
               )

In [ ]:
start = time.time()
history = LR_plus.fit(x_data, y_data, batch_size=16, epochs=10, verbose =1)
end = time.time()
print(end-start)

In [ ]:
cm = confusion_matrix(y_data, np.argmax(LR_plus(x_data).numpy(),axis=-1))

In [ ]:
cm

In [ ]:
x_axis = range(10)
fig, ax = plt.subplots(figsize=(12,6), nrows=1, ncols=2, sharex=True, sharey=False)
ax = ax.flatten()

loss = history.history['loss']
ax[0].plot(x_axis, loss)
ax[0].set_title('softmax classifier loss curve')

acc  = history.history['acc']
ax[1].plot(x_axis, acc)
ax[1].set_title('softmax classifier acc  curve')


ax[0].set_ylabel('loss')
ax[1].set_ylabel('acc')

In [ ]:
12214/12476